# SQL2Circuits development notebook

In [1]:
import os
from data_preparation.queries import QueryGenerator
from data_preparation.prepare import DataPreparation
from data_preparation.database import Database
from circuit_preparation.circuits import Circuits
import os
from training.train import SQL2CircuitsTrainer


this_folder = os.path.abspath(os.getcwd())
seed_paths = ["data_preparation//query_seeds//JOB_query_seed_execution_time.json",
              "data_preparation//query_seeds//JOB_query_seed_cardinality.json"]
workload_types = ["E", "C"]

run_id = 1
ty = 1
workload_type = workload_types[ty]
database = Database("IMDB")
generator = QueryGenerator(run_id, database = "IMDB", workload_type = workload_type, query_seed_file_path = seed_paths[ty])

query_file = generator.get_query_file()
data_preparator = DataPreparation(run_id, query_file, database = database, workload_type = workload_type)


Number of training queries is  398
Number of test queries is  140
Number of validation queries is  135
Error while fetching data from PostgreSQL connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



In [2]:
this_folder = os.path.abspath(os.getcwd())
output_folder = this_folder + "//circuit_preparation//data//circuits//" + str(run_id) + "//"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print("The new directory: ", output_folder, " is created.")

circuits = Circuits(run_id, query_file, output_folder, write_cfg_to_file = True, write_pregroup_to_file=True, generate_circuit_png_diagrams = True)
circuits.execute_full_transformation()


In [3]:
trainer = SQL2CircuitsTrainer(run_id, circuits, data_preparator, workload = "cardinality", classification = 2)
trainer.train()

id: 1
a: 0.01
c: 0.01
epochs: 1000
initial_number_of_circuits: 20
classification: 4
workload: cardinality
optimization_medthod: SPSA



Initializing new parameters
Progress:  0.052


number_of_training_circuits: 21
number_of_validation_circuits: 51
number_of_test_circuits: 57
number_of_parameters_in_model: 138



NameError: free variable 'costs' referenced before assignment in enclosing scope